# Checking file path of input data

In [1]:
import os
import glob

# Define paths to the folders
image_folder = r"C:\Users\ISU\cv_dataset\image_data"
segmentation_folder = r"C:\Users\ISU\cv_dataset\seg_data"

# Get all files and sort them
image_files = sorted(glob.glob(os.path.join(image_folder, "*.nii")))
seg_files = sorted(glob.glob(os.path.join(segmentation_folder, "*.nii")))

# Ensure they are matched correctly
print("Image files:", image_files)
print("Segmentation files:", seg_files)


Image files: ['C:\\Users\\ISU\\cv_dataset\\image_data\\pat0_cropped_norm.nii', 'C:\\Users\\ISU\\cv_dataset\\image_data\\pat1_cropped_norm.nii', 'C:\\Users\\ISU\\cv_dataset\\image_data\\pat2_cropped_norm.nii']
Segmentation files: ['C:\\Users\\ISU\\cv_dataset\\seg_data\\pat0_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\seg_data\\pat1_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\seg_data\\pat2_cropped_seg.nii']


# Model produces output file without padding

In [10]:
import glob
import os
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from PIL import Image

# Correct paths
image_folder = r"C:\Users\ISU\cv_dataset\open_source_data\preprocessed"
segmentation_folder = r"C:\Users\ISU\cv_dataset\open_source_data\preprocessed"

# Find files using specific naming convention
image_files = sorted(glob.glob(os.path.join(image_folder, "preprocessed_pat*_cropped_norm.nii")))
seg_files = sorted(glob.glob(os.path.join(segmentation_folder, "preprocessed_mask_pat*_cropped_seg.nii")))

print("Image files:", image_files)
print("Segmentation files:", seg_files)

# Directories for saving preprocessed data
open_source_dir = "open_source_data"
patient_dir = "patient_data"
os.makedirs(open_source_dir, exist_ok=True)
os.makedirs(patient_dir, exist_ok=True)

# Configurations
input_shape = (128, 128, 64)  # Resize dimensions
batch_size = 2
epochs = 20
learning_rate = 1e-4

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1: Preprocess the Open-Source Dataset ###
def preprocess_open_source_data(image_paths, seg_paths, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for img_path, seg_path in zip(image_paths, seg_paths):
        # Load image and segmentation
        img = nib.load(img_path).get_fdata()
        seg = nib.load(seg_path).get_fdata()

        # Convert segmentation to binary mask
        binary_seg = np.isin(seg, [1, 2, 3, 4, 5, 6, 7, 8]).astype(np.uint8)

        # Save preprocessed image and mask
        nib.save(nib.Nifti1Image(img, np.eye(4)), os.path.join(output_dir, f"preprocessed_{os.path.basename(img_path)}"))
        nib.save(nib.Nifti1Image(binary_seg, np.eye(4)), os.path.join(output_dir, f"preprocessed_mask_{os.path.basename(seg_path)}"))

# Preprocess the open-source data
preprocessed_data_dir = os.path.join(open_source_dir, "preprocessed")
preprocess_open_source_data(image_files, seg_files, preprocessed_data_dir)

### Step 2: Preprocess the Patient File ###
def preprocess_patient_data(patient_img_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Load and normalize patient image
    patient_img = nib.load(patient_img_path).get_fdata()
    patient_img = (patient_img - patient_img.min()) / (patient_img.max() - patient_img.min())

    # Save preprocessed patient image
    preprocessed_patient_path = os.path.join(output_dir, "preprocessed_patient_image.nii.gz")
    nib.save(nib.Nifti1Image(patient_img, np.eye(4)), preprocessed_patient_path)

    return preprocessed_patient_path

# Example path for patient image
patient_img_path = r"C:\Users\ISU\cv_dataset\patient_image.nii"  # Replace with actual path
preprocessed_patient_path = preprocess_patient_data(patient_img_path, patient_dir)

### Step 3: Define Dataset and Dataloader ###
import scipy.ndimage

class HeartDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def resize_3d(self, img, shape):
        zoom_factors = [shape[0] / img.shape[0], shape[1] / img.shape[1], shape[2] / img.shape[2]]
        resized_img = scipy.ndimage.zoom(img, zoom_factors, order=1)
        return resized_img

    def __getitem__(self, idx):
        img = nib.load(self.image_paths[idx]).get_fdata()
        mask = nib.load(self.mask_paths[idx]).get_fdata()

        # Normalize image
        img = (img - img.min()) / (img.max() - img.min())
        
        # Resize the image and mask
        img = self.resize_3d(img, input_shape)
        mask = self.resize_3d(mask, input_shape)

        # Convert to 3D format for PyTorch
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension (1 for grayscale images)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # Add channel dimension (1 for binary masks)

        # Apply transform if available
        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)

        return img, mask



# Load paths for preprocessed data
image_paths = sorted(glob.glob(os.path.join(preprocessed_data_dir, "preprocessed_pat*_cropped_norm.nii")))
mask_paths = sorted(glob.glob(os.path.join(preprocessed_data_dir, "preprocessed_mask_pat*_cropped_seg.nii")))

# Split into training and validation
train_img, val_img, train_mask, val_mask = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

# Define transformations
transform = transforms.Compose([
])


# Create datasets and dataloaders
train_dataset = HeartDataset(train_img, train_mask, transform=transform)
val_dataset = HeartDataset(val_img, val_mask, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Step 4: Define U-Net Model ###

class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        self.upconv = nn.ConvTranspose3d(64, 64, kernel_size=2, stride=2)

    def forward(self, x):
        enc = self.encoder(x)
        dec = self.upconv(enc)
        dec = self.decoder(dec)
        return dec

# Initialize the model
model = UNet3D(in_channels=1, out_channels=1).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.BCELoss()

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        images, masks = batch
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

torch.save(model.state_dict(), "unet_heart_segmentation_3d.pth")


### Step 5: Evaluate on Patient File ###
def segment_patient_image(model, patient_img_path, output_path):
    model.eval()
    patient_img = nib.load(patient_img_path).get_fdata()
    patient_img = np.expand_dims(patient_img, axis=0)  # Add channel dimension
    patient_img = np.expand_dims(patient_img, axis=0)  # Add batch dimension
    patient_img = torch.tensor(patient_img, dtype=torch.float32).to(device)

    with torch.no_grad():
        prediction = model(patient_img)
        prediction = torch.sigmoid(prediction).cpu().numpy().squeeze()

    # Save segmentation result
    nib.save(nib.Nifti1Image(prediction, np.eye(4)), output_path)
    print(f"Segmented heart saved at: {output_path}")

segmented_heart_path = os.path.join(patient_dir, "segmented_heart.nii.gz")
segment_patient_image(model, preprocessed_patient_path, segmented_heart_path)


Image files: ['C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_pat0_cropped_norm.nii', 'C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_pat1_cropped_norm.nii', 'C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_pat2_cropped_norm.nii']
Segmentation files: ['C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_mask_pat0_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_mask_pat1_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\open_source_data\\preprocessed\\preprocessed_mask_pat2_cropped_seg.nii']
Epoch 1/20, Loss: 0.7030
Epoch 2/20, Loss: 0.6994
Epoch 3/20, Loss: 0.6960
Epoch 4/20, Loss: 0.6927
Epoch 5/20, Loss: 0.6893
Epoch 6/20, Loss: 0.6858
Epoch 7/20, Loss: 0.6821
Epoch 8/20, Loss: 0.6781
Epoch 9/20, Loss: 0.6738
Epoch 10/20, Loss: 0.6691
Epoch 11/20, Loss: 0.6641
Epoch 12/20, Loss: 0.6586
Epoch 13/20, Loss: 0.6526
Epoch 14/20, Loss: 0.6462
Epoch 15/2

# Model produces output file with padding

In [11]:
import os
import glob
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import scipy.ndimage

# Set base directories
BASE_DIR = r"C:\Users\ISU\cv_dataset"
IMAGE_FOLDER = os.path.join(BASE_DIR, "open_source_data", "preprocessed")
SEGMENTATION_FOLDER = IMAGE_FOLDER  # Assuming images and masks are in the same folder
PATIENT_IMAGE_PATH = os.path.join(BASE_DIR, "patient_image.nii")  # Replace with actual patient file

# File matching patterns
IMAGE_PATTERN = "preprocessed_pat*_cropped_norm.nii"
MASK_PATTERN = "preprocessed_mask_pat*_cropped_seg.nii"

# Get image and segmentation file paths
image_files = sorted(glob.glob(os.path.join(IMAGE_FOLDER, IMAGE_PATTERN)))
segmentation_files = sorted(glob.glob(os.path.join(SEGMENTATION_FOLDER, MASK_PATTERN)))

if not image_files or not segmentation_files:
    raise FileNotFoundError("No image or segmentation files found. Please check the paths or file naming conventions.")

print(f"Found {len(image_files)} images and {len(segmentation_files)} segmentations.")

# Output directories
OUTPUT_DIR_OPEN_SOURCE = os.path.join(BASE_DIR, "open_source_data")
OUTPUT_DIR_PATIENT = os.path.join(BASE_DIR, "patient_data")
os.makedirs(OUTPUT_DIR_OPEN_SOURCE, exist_ok=True)
os.makedirs(OUTPUT_DIR_PATIENT, exist_ok=True)

# Configurations
INPUT_SHAPE = (128, 128, 64)  # Resize dimensions
BATCH_SIZE = 2
EPOCHS = 20
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Step 1: Preprocess Open-Source Data ###
def preprocess_open_source_data(image_paths, mask_paths, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for img_path, seg_path in zip(image_paths, mask_paths):
        # Load image and segmentation
        img = nib.load(img_path).get_fdata()
        seg = nib.load(seg_path).get_fdata()

        # Convert segmentation to binary mask
        binary_seg = np.isin(seg, range(1, 9)).astype(np.uint8)  # Map values 1-8 to binary mask

        # Save preprocessed data
        img_name = f"preprocessed_{os.path.basename(img_path)}"
        seg_name = f"preprocessed_mask_{os.path.basename(seg_path)}"
        nib.save(nib.Nifti1Image(img, np.eye(4)), os.path.join(output_dir, img_name))
        nib.save(nib.Nifti1Image(binary_seg, np.eye(4)), os.path.join(output_dir, seg_name))

    print(f"Preprocessed open-source data saved in: {output_dir}")


preprocess_open_source_data(image_files, segmentation_files, OUTPUT_DIR_OPEN_SOURCE)


### Step 2: Preprocess Patient Image ###
def preprocess_patient_data(patient_img_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Load and normalize patient image
    patient_img = nib.load(patient_img_path).get_fdata()
    patient_img = (patient_img - patient_img.min()) / (patient_img.max() - patient_img.min())

    # Save preprocessed patient image
    preprocessed_patient_path = os.path.join(output_dir, "preprocessed_patient_image.nii.gz")
    nib.save(nib.Nifti1Image(patient_img, np.eye(4)), preprocessed_patient_path)

    print(f"Preprocessed patient image saved in: {preprocessed_patient_path}")
    return preprocessed_patient_path


preprocessed_patient_path = preprocess_patient_data(PATIENT_IMAGE_PATH, OUTPUT_DIR_PATIENT)


### Step 3: Dataset Definition ###
class HeartDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def resize_3d(self, img, shape):
        zoom_factors = [shape[i] / img.shape[i] for i in range(3)]
        return scipy.ndimage.zoom(img, zoom_factors, order=1)

    def __getitem__(self, idx):
        img = nib.load(self.image_paths[idx]).get_fdata()
        mask = nib.load(self.mask_paths[idx]).get_fdata()

        # Normalize image
        img = (img - img.min()) / (img.max() - img.min())

        # Resize the image and mask
        img = self.resize_3d(img, INPUT_SHAPE)
        mask = self.resize_3d(mask, INPUT_SHAPE)

        # Convert to PyTorch tensors
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return img, mask


# Split into train and validation sets
train_img, val_img, train_mask, val_mask = train_test_split(image_files, segmentation_files, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = HeartDataset(train_img, train_mask)
val_dataset = HeartDataset(val_img, val_mask)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


### Step 4: Define U-Net Model ###
class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        self.upconv = nn.ConvTranspose3d(64, 64, kernel_size=2, stride=2)

    def forward(self, x):
        enc = self.encoder(x)
        dec = self.upconv(enc)
        dec = self.decoder(dec)
        return dec


model = UNet3D(in_channels=1, out_channels=1).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.BCELoss()

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

torch.save(model.state_dict(), os.path.join(BASE_DIR, "unet_heart_segmentation_3d.pth"))


### Step 5: Segment Patient Image ###
def segment_patient_image(model, patient_img_path, output_path):
    model.eval()
    patient_img = nib.load(patient_img_path).get_fdata()
    patient_img = np.expand_dims(patient_img, axis=(0, 1))  # Add channel and batch dimensions
    patient_img = torch.tensor(patient_img, dtype=torch.float32).to(DEVICE)

    with torch.no_grad():
        prediction = model(patient_img)
        prediction = torch.sigmoid(prediction).cpu().numpy().squeeze()

    nib.save(nib.Nifti1Image(prediction, np.eye(4)), output_path)
    print(f"Segmented heart saved at: {output_path}")


segmented_heart_path = os.path.join(OUTPUT_DIR_PATIENT, "segmented_heart.nii.gz")
segment_patient_image(model, preprocessed_patient_path, segmented_heart_path)


Found 3 images and 3 segmentations.
Preprocessed open-source data saved in: C:\Users\ISU\cv_dataset\open_source_data
Preprocessed patient image saved in: C:\Users\ISU\cv_dataset\patient_data\preprocessed_patient_image.nii.gz
Epoch 1/20, Loss: 0.6961
Epoch 2/20, Loss: 0.6939
Epoch 3/20, Loss: 0.6917
Epoch 4/20, Loss: 0.6895
Epoch 5/20, Loss: 0.6873
Epoch 6/20, Loss: 0.6850
Epoch 7/20, Loss: 0.6826
Epoch 8/20, Loss: 0.6800
Epoch 9/20, Loss: 0.6772
Epoch 10/20, Loss: 0.6741
Epoch 11/20, Loss: 0.6707
Epoch 12/20, Loss: 0.6670
Epoch 13/20, Loss: 0.6628
Epoch 14/20, Loss: 0.6582
Epoch 15/20, Loss: 0.6530
Epoch 16/20, Loss: 0.6474
Epoch 17/20, Loss: 0.6412
Epoch 18/20, Loss: 0.6345
Epoch 19/20, Loss: 0.6273
Epoch 20/20, Loss: 0.6194
Segmented heart saved at: C:\Users\ISU\cv_dataset\patient_data\segmented_heart.nii.gz
